In [1]:
!pip install keras-tuner

In [3]:
import os
import numpy as np
import wfdb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import mlflow
import mlflow.tensorflow
import keras_tuner as kt
from datetime import datetime

# --- Configuration ---
data_path = "mitbih"  # Replace with your actual path
mlflow.set_tracking_uri("http://127.0.0.1:5002/")

# --- Record List ---
record_ids = ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115',
              '116', '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207', '208',
              '209', '210', '212', '213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231',
              '232', '233', '234']

# --- Data Loading ---
print("Loading ECG data...")
X = []
y = []
sample_record_ids = []

for rec_id in record_ids:
    try:
        record = wfdb.rdrecord(os.path.join(data_path, rec_id))
        annotation = wfdb.rdann(os.path.join(data_path, rec_id), 'atr')

        signal = record.p_signal[:, 0]  # Lead I
        ann_samples = annotation.sample
        ann_symbols = annotation.symbol

        for i, sample in enumerate(ann_samples):
            if sample - 90 < 0 or sample + 90 > len(signal):
                continue
            beat_segment = signal[sample - 90: sample + 90]
            label = 0 if ann_symbols[i] == 'N' else 1

            X.append(beat_segment)
            y.append(label)
            sample_record_ids.append(rec_id)

    except Exception as e:
        print(f"Skipping record {rec_id}: {e}")

X = np.array(X).reshape(-1, 180, 1)
y = np.array(y)
sample_record_ids = np.array(sample_record_ids)

print(f"Total samples loaded: {len(X)}")
print(f"Class distribution - Normal: {np.sum(y == 0)}, Abnormal: {np.sum(y == 1)}")

# --- Convert record ID to index (0 to 47) ---
record_id_to_index = {rec: idx for idx, rec in enumerate(record_ids)}
sample_record_indices = np.array([record_id_to_index[rid] for rid in sample_record_ids])

# --- Train/Val/Test Split (60/20/20) ---
np.random.seed(42)
all_indices = np.arange(len(record_ids))
np.random.shuffle(all_indices)
train_ids = all_indices[:29]
val_ids = all_indices[29:39]
test_ids = all_indices[39:]

X_train = X[np.isin(sample_record_indices, train_ids)]
y_train = y[np.isin(sample_record_indices, train_ids)]

X_val = X[np.isin(sample_record_indices, val_ids)]
y_val = y[np.isin(sample_record_indices, val_ids)]

X_test = X[np.isin(sample_record_indices, test_ids)]
y_test = y[np.isin(sample_record_indices, test_ids)]

print(f"Train samples: {len(X_train)}, Val samples: {len(X_val)}, Test samples: {len(X_test)}")

# --- Hyperparameter Tuning Model Builder ---
def build_model(hp):
    """Build model with hyperparameters to tune"""
    model = Sequential()
    
    # First Conv1D layer
    model.add(Conv1D(
        filters=hp.Choice('conv1_filters', values=[16, 32, 64]),
        kernel_size=hp.Choice('conv1_kernel', values=[3, 5, 7]),
        activation='relu',
        input_shape=(180, 1)
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2))
    
    # Second Conv1D layer
    model.add(Conv1D(
        filters=hp.Choice('conv2_filters', values=[32, 64, 128]),
        kernel_size=hp.Choice('conv2_kernel', values=[3, 5, 7]),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2))
    
    # Optional third Conv1D layer
    if hp.Boolean('add_conv3'):
        model.add(Conv1D(
            filters=hp.Choice('conv3_filters', values=[64, 128, 256]),
            kernel_size=hp.Choice('conv3_kernel', values=[3, 5]),
            activation='relu'
        ))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(2))
    
    model.add(Flatten())
    
    # Dense layers
    model.add(Dense(
        units=hp.Choice('dense_units', values=[32, 64, 128, 256]),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.7, step=0.1)))
    
    # Optional second dense layer
    if hp.Boolean('add_dense2'):
        model.add(Dense(
            units=hp.Choice('dense2_units', values=[16, 32, 64]),
            activation='relu'
        ))
        model.add(Dropout(hp.Float('dropout2_rate', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# --- Custom Tuner with MLflow Integration ---
class MLflowTuner(kt.RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        # Start MLflow run for this trial
        with mlflow.start_run(run_name=f"trial_{trial.trial_id}", nested=True):
            # Log hyperparameters
            hp_values = trial.hyperparameters.values
            for param, value in hp_values.items():
                mlflow.log_param(param, value)
            
            # Run the trial
            result = super().run_trial(trial, *args, **kwargs)
            
            # Log the best validation accuracy for this trial
            if hasattr(result, 'history') and 'val_accuracy' in result.history:
                best_val_acc = max(result.history['val_accuracy'])
                mlflow.log_metric("best_val_accuracy", best_val_acc)
                mlflow.log_metric("final_val_accuracy", result.history['val_accuracy'][-1])
            
            return result

# --- Hyperparameter Tuning ---
print("Starting hyperparameter tuning...")

# Create tuner
tuner = MLflowTuner(
    build_model,
    objective=kt.Objective('val_accuracy', direction='max'),
    max_trials=20,  # Adjust based on your computational resources
    directory='ecg_tuning',
    project_name='ecg_cnn_hyperparameter_search',
    overwrite=True
)

# Start parent MLflow run
with mlflow.start_run(run_name="ECG_Hyperparameter_Tuning_Experiment"):
    mlflow.log_param("total_trials", 20)
    mlflow.log_param("tuning_objective", "val_accuracy")
    mlflow.log_param("train_records", len(train_ids))
    mlflow.log_param("val_records", len(val_ids))
    mlflow.log_param("test_records", len(test_ids))
    
    # Search for best hyperparameters
    tuner.search(
        X_train, y_train,
        epochs=15,  # Reduced epochs for faster tuning
        batch_size=128,
        validation_data=(X_val, y_val),
        verbose=1,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
    )
    
    # Get best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    
    print("\n" + "="*50)
    print("BEST HYPERPARAMETERS FOUND:")
    print("="*50)
    
    best_hp_dict = {}
    for param in ['conv1_filters', 'conv1_kernel', 'conv2_filters', 'conv2_kernel', 
                  'add_conv3', 'dense_units', 'dropout_rate', 'add_dense2', 'learning_rate']:
        if param in best_hps.values:
            value = best_hps.get(param)
            best_hp_dict[param] = value
            print(f"{param}: {value}")
            mlflow.log_param(f"best_{param}", value)
    
    # Additional conv3 parameters if enabled
    if best_hps.get('add_conv3'):
        for param in ['conv3_filters', 'conv3_kernel']:
            if param in best_hps.values:
                value = best_hps.get(param)
                best_hp_dict[param] = value
                print(f"{param}: {value}")
                mlflow.log_param(f"best_{param}", value)
    
    # Additional dense2 parameters if enabled
    if best_hps.get('add_dense2'):
        for param in ['dense2_units', 'dropout2_rate']:
            if param in best_hps.values:
                value = best_hps.get(param)
                best_hp_dict[param] = value
                print(f"{param}: {value}")
                mlflow.log_param(f"best_{param}", value)
    
    print("="*50)
    
    # --- Train Final Model with Best Hyperparameters ---
    print("\nTraining final model with best hyperparameters...")
    
    # Build and train the best model with more epochs
    best_model = tuner.hypermodel.build(best_hps)
    
    # Train with best hyperparameters
    history = best_model.fit(
        X_train, y_train,
        epochs=25,  # More epochs for final training
        batch_size=128,
        validation_data=(X_val, y_val),
        verbose=1,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5, min_lr=1e-7)
        ]
    )
    
    # --- Final Evaluation ---
    print("\nEvaluating final model...")
    
    # Evaluate on test set
    test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
    
    # Log final metrics
    mlflow.log_metric("final_test_loss", test_loss)
    mlflow.log_metric("final_test_accuracy", test_accuracy)
    mlflow.log_metric("final_train_accuracy", max(history.history['accuracy']))
    mlflow.log_metric("final_val_accuracy", max(history.history['val_accuracy']))
    
    print(f"\nFINAL RESULTS:")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Best Validation Accuracy: {max(history.history['val_accuracy']):.4f}")
    
    # Predictions and detailed evaluation
    y_pred_proba = best_model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Classification Report
    report = classification_report(y_test, y_pred)
    print(f"\nClassification Report:\n{report}")
    
    with open("best_model_classification_report.txt", "w") as f:
        f.write("BEST HYPERPARAMETERS:\n")
        f.write("="*50 + "\n")
        for param, value in best_hp_dict.items():
            f.write(f"{param}: {value}\n")
        f.write("\n" + "="*50 + "\n")
        f.write("CLASSIFICATION REPORT:\n")
        f.write("="*50 + "\n")
        f.write(report)
    mlflow.log_artifact("best_model_classification_report.txt")
    
    # Confusion Matrix
    conf_mat = confusion_matrix(y_test, y_pred)
    print(f"\nConfusion Matrix:\n{conf_mat}")
    
    with open("best_model_confusion_matrix.txt", "w") as f:
        f.write("CONFUSION MATRIX:\n")
        f.write(np.array2string(conf_mat))
    mlflow.log_artifact("best_model_confusion_matrix.txt")
    
    # Plot Training History
    plt.figure(figsize=(15, 5))
    
    # Accuracy plot
    plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
    plt.title('Model Accuracy (Best Hyperparameters)')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    # Loss plot
    plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Train Loss', marker='o')
    plt.plot(history.history['val_loss'], label='Validation Loss', marker='s')
    plt.title('Model Loss (Best Hyperparameters)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Learning rate plot (if available)
    plt.subplot(1, 3, 3)
    if 'lr' in history.history:
        plt.plot(history.history['lr'], label='Learning Rate', marker='d')
        plt.title('Learning Rate Schedule')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.yscale('log')
        plt.legend()
        plt.grid(True)
    else:
        plt.text(0.5, 0.5, 'Learning Rate\nNot Tracked', 
                ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('Learning Rate')
    
    plt.tight_layout()
    plt.savefig("best_model_training_history.png", dpi=300, bbox_inches='tight')
    mlflow.log_artifact("best_model_training_history.png")
    plt.close()
    
    # Save the best model
    best_model.save("best_ecg_model.h5")
    mlflow.log_artifact("best_ecg_model.h5")
    
    # Summary of tuning results
    tuning_summary = f"""
HYPERPARAMETER TUNING SUMMARY
==============================
Total Trials: 20
Best Validation Accuracy: {max(history.history['val_accuracy']):.4f}
Final Test Accuracy: {test_accuracy:.4f}

Best Hyperparameters:
{'-'*30}
"""
    for param, value in best_hp_dict.items():
        tuning_summary += f"{param}: {value}\n"
    
    with open("hyperparameter_tuning_summary.txt", "w") as f:
        f.write(tuning_summary)
    mlflow.log_artifact("hyperparameter_tuning_summary.txt")
    
    print("\n" + "="*60)
    print("HYPERPARAMETER TUNING COMPLETED!")
    print(f"Best model saved with test accuracy: {test_accuracy:.4f}")
    print("Check MLflow UI for detailed experiment tracking")
    print("="*60)

Trial 20 Complete [00h 03m 36s]
val_accuracy: 0.7977863550186157

Best val_accuracy So Far: 0.8779886960983276
Total elapsed time: 00h 32m 52s

BEST HYPERPARAMETERS FOUND:
conv1_filters: 16
conv1_kernel: 5
conv2_filters: 32
conv2_kernel: 7
add_conv3: False
dense_units: 32
dropout_rate: 0.6000000000000001
add_dense2: False
learning_rate: 0.007752323623504601

Training final model with best hyperparameters...
Epoch 1/25
546/546 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9055 - loss: 0.2915 - val_accuracy: 0.8052 - val_loss: 0.5107 - learning_rate: 0.0078
Epoch 2/25
546/546 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9609 - loss: 0.1235 - val_accuracy: 0.7834 - val_loss: 0.6172 - learning_rate: 0.0078
Epoch 3/25
546/546 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9679 - loss: 0.1021 - val_accuracy: 0.8068 - val_loss: 0.6494 - learning_rate: 0.0078
Epoch 4/25
546/546 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9680 - loss: 0.1014 - val_accuracy: 0.8453 - val_loss: 0.3986 -


HYPERPARAMETER TUNING COMPLETED!
Best model saved with test accuracy: 0.8199
Check MLflow UI for detailed experiment tracking
🏃 View run ECG_Hyperparameter_Tuning_Experiment at: http://127.0.0.1:5002/#/experiments/0/runs/96744611c57a472ab621c49edc814ccd
🧪 View experiment at: http://127.0.0.1:5002/#/experiments/0
